In [1]:
import pandas as pd 
import numpy as np
import os
from functools import reduce
from sklearn.preprocessing import MinMaxScaler
# import utils as utils located as src/utils/utils.py
from src.utils import utils, gcp_utils as gutils
from src.api import data_requests as dr
from src.processing import calculate_qb_metrics as cqm, calculate_defense_metrics as cdm, score_defense_metrics as sdm, score_qb_metrics as sqm
from src.processing import score_qb_metrics as sqm
from concurrent.futures import ThreadPoolExecutor, as_completed
from retry import retry
import functools
from multiprocessing import Pool
from google.cloud import firestore

API_KEY = utils.read_api_key()
import requests
BASE_URL = "https://api.collegefootballdata.com"
HEADERS = {"Authorization": f"Bearer {API_KEY}"}
team = 'Michigan'
year=2022


In [4]:
def get_qb_by_name(qb_name, collection_name="qb_overall_rankings_2022"):
    """
    Queries Firestore collection "qb_overall_rankings_2022" and returns the quarterback with the specified name.

    Args:
        qb_name (str): The name of the quarterback to search for.
        collection_name (str): The name of the Firestore collection containing the QB data.

    Returns:
        dict: Dictionary containing the QB data if found, or an error message if not found.
    """
    try:
        # Initialize Firestore client
        db = gutils.get_firestore_client_db()

        # Query the Firestore collection for the specified quarterback
        query = db.collection(collection_name).where("player", "==", qb_name).stream()

        # Retrieve the document from the query
        qb_data = [doc.to_dict() for doc in query]

        # Check if the quarterback is found
        if len(qb_data) == 0:
            return {"success": False, "errors": [f"No quarterback found with the name '{qb_name}'"], "data": []}

        return qb_data[0]

    except Exception as e:
        print(f"Error fetching quarterback by name: {e}")
        return {"success": False, "errors": [f"Error fetching quarterback by name: {e}"], "data": []}


In [5]:
get_qb_by_name('Anthony Richardson')

{'year': 2022,
 'total_score_rank': 20.0,
 'avg_qb_total_score': 82.15962102529328,
 'player': 'Anthony Richardson',
 'competitive_score_rank': 17.0,
 'avg_qb_competitive_score': 80.67083018897867}

In [6]:
qbs

NameError: name 'qbs' is not defined

In [7]:
# qbs = gutils.query_firestore_general('meta', 'qbs')
from src.internal_api import queries as iq

ar = iq.query_firestore_player_data('Anthony Richardson', year)

/opt/homebrew/Caskroom/miniforge/base/envs/PowerRank/lib/python3.9/site-packages/google/cloud/firestore_v1/base_collection.py:290: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)
/Users/djschor/Projects/ncaafb_power_rank/src/internal_api/queries.py:122: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  query = collection_ref.where(u'player', '==', player_name).where(u'year', u'==', year)


Successfully queried qb_weekly_rankings_2022 for player Anthony Richardson and year 2022


In [9]:
given this array of objects called playerData that looks like this with player performance for each week of season 
[{'aqs': 50.297619047619044, 'qpi_relative_rank': 6.0, 'ppi_relative_rank': 3.0, 'aqs_competitive_rank': 70.0, 'ppi_competitive_rank': 49.0, 'reer_competitive_rank': 35.0, 'adpsr': 8.823529411764707, 'ppi': 49.58027315144535, 'qb_total_rank': 430.0, 'adpsr_relative_rank': 4.0, 'aqs_relative_rank': 6.0, 'total_score_week_rank': 46.0, 'dmi_relative_rank': 10.0, 'sei_relative_rank': 4.0, 'qb_relative_score': 82.83492738262092, 'competitive_score_week_rank': 38.0, 'qb_total_score': 85.26650842380342, 'reer': 37.5, 'team': 'Florida', 'dmi': 78.94736842105263, 'crae_competitive_rank': 207.0, 'qb_competitive_score': 82.21352686238725, 'reer_relative_rank': 3.0, 'qpi': 196.42857142857142, 'sei_competitive_rank': 35.0, 'year': 2022, 'defense_score_competitive_rank': 54.0, 'dmi_competitive_rank': 74.0, 'week': 10, 'crae_relative_rank': 9.0, 'defense_score_relative_rank': 3.0, 'crae': -19.685847635292454, 'defense_score': 55.85457407288959, 'qpi_competitive_rank': 64.0, 'pk': 'Anthony Richardson_10_2022', 'adpsr_competitive_rank': 28.0, 'player': 'Anthony Richardson', 'sei': 766.6666666666667}, {'defense_score_relative_rank': 4.0, 'crae_competitive_rank': 227.0, 'reer': 33.33333333333333, 'year': 2022, 'qb_total_score': 80.89283281666376, 'aqs': 47.46376811594203, 'qb_competitive_score': 80.37007742963226, 'aqs_competitive_rank': 65.0, 'team': 'Florida', 'qpi_relative_rank': 11.0, 'crae_relative_rank': 8.0, 'dmi_relative_rank': 12.0, 'ppi': 45.67826751537143, 'sei_competitive_rank': 18.0, 'crae': -18.000726425852264, 'aqs_relative_rank': 8.0, 'week': 11, 'reer_competitive_rank': 46.0, 'qpi_competitive_rank': 114.0, 'reer_relative_rank': 7.0, 'total_score_week_rank': 77.0, 'defense_score': 54.61947841044724, 'ppi_relative_rank': 10.0, 'qb_relative_score': 78.41480570350281, 'qb_total_rank': 976.0, 'sei_relative_rank': 3.0, 'adpsr': 3.4482758620689653, 'sei': 981.8181818181819, 'competitive_score_week_rank': 70.0, 'dmi_competitive_rank': 116.0, 'adpsr_competitive_rank': 72.0, 'pk': 'Anthony Richardson_11_2022', 'qpi': 135.86956521739134, 'defense_score_competitive_rank': 26.0, 'player': 'Anthony Richardson', 'dmi': 57.14285714285714, 'ppi_competitive_rank': 121.0, 'adpsr_relative_rank': 9.0}, ...]
give me a react typescript function that given one of the keys as input (i.e, aqs_competitive_rank), returns a list of those values for each week of the input array 

[{'aqs': 50.297619047619044, 'qpi_relative_rank': 6.0, 'ppi_relative_rank': 3.0, 'aqs_competitive_rank': 70.0, 'ppi_competitive_rank': 49.0, 'reer_competitive_rank': 35.0, 'adpsr': 8.823529411764707, 'ppi': 49.58027315144535, 'qb_total_rank': 430.0, 'adpsr_relative_rank': 4.0, 'aqs_relative_rank': 6.0, 'total_score_week_rank': 46.0, 'dmi_relative_rank': 10.0, 'sei_relative_rank': 4.0, 'qb_relative_score': 82.83492738262092, 'competitive_score_week_rank': 38.0, 'qb_total_score': 85.26650842380342, 'reer': 37.5, 'team': 'Florida', 'dmi': 78.94736842105263, 'crae_competitive_rank': 207.0, 'qb_competitive_score': 82.21352686238725, 'reer_relative_rank': 3.0, 'qpi': 196.42857142857142, 'sei_competitive_rank': 35.0, 'year': 2022, 'defense_score_competitive_rank': 54.0, 'dmi_competitive_rank': 74.0, 'week': 10, 'crae_relative_rank': 9.0, 'defense_score_relative_rank': 3.0, 'crae': -19.685847635292454, 'defense_score': 55.85457407288959, 'qpi_competitive_rank': 64.0, 'pk': 'Anthony Richardson_

In [1]:
# import queries 
from src.internal_api import queries as iq
iq.query_firestore_player_data('', '', 'Anthony Richardson', 2022)

ModuleNotFoundError: No module named 'flask_sqlalchemy'

In [ ]:
    gutils.save_data_to_firestore_id({'qbs': qb_names}, gutils.get_firestore_client_db(), 'meta', 'qbs')


In [ ]:
def get_top_qbs_by_week(week, top_x):
    collection_name = "qb_weekly_rankings_2022"

    # Initialize Firestore client
    db = gutils.get_firestore_client_db()

    # Query the Firestore collection for documents with the specified week and order by qb_total_score
    query = db.collection(collection_name).where("week", "==", week).order_by("qb_total_score", direction=firestore.Query.DESCENDING).limit(top_x).stream()

    # Retrieve all documents from the query
    top_qbs = [doc.to_dict() for doc in query]

    return top_qbs

def get_top_qbs_overall(top_x, field='avg_qb_total_score'):
    if field not in {'avg_qb_competitive_score', 'avg_qb_total_score', 'competitive_score_rank', 'total_score_rank'}:
        raise ValueError("Invalid field. Valid fields are: 'avg_qb_competitive_score', 'avg_qb_total_score', 'competitive_score_rank', and 'total_score_rank'.")

    collection_name = "qb_overall_rankings_2022"

    # Initialize Firestore client
    db = gutils.get_firestore_client_db()

    # Query the Firestore collection, order by the specified field, and limit the results to top X
    query = db.collection(collection_name).order_by(field, direction=firestore.Query.DESCENDING if field.startswith('avg_') else firestore.Query.ASCENDING).limit(top_x).stream()

    # Retrieve all documents from the query
    top_qbs = [doc.to_dict() for doc in query]

    return top_qbs

In [ ]:
from ariadne import load_schema_from_path, make_executable_schema, \
    graphql_sync, snake_case_fallback_resolvers, ObjectType
from ariadne.constants import PLAYGROUND_HTML
from flask import request, jsonify
import pandas as pd 
import numpy as np
import os
from functools import reduce
from sklearn.preprocessing import MinMaxScaler
from concurrent.futures import ThreadPoolExecutor, as_completed
from retry import retry
import functools
from multiprocessing import Pool
from google.cloud import firestore

In [23]:
def query_firestore_player_data(player_name, year):
    """
    Queries the 'qb_weekly_rankings_2022' collection in Firestore to get quarterback weekly performance data
    for the specified player name and year.

    Args:
        player_name (str): The name of the quarterback.
        year (int): The year of the quarterback's performance data.

    Returns:
        list: A list of dictionaries representing the documents with the specified player name and year if any exist, None otherwise.
    """
    try:
        collection_name = 'qb_weekly_rankings_2022'

        # Initialize Firestore client
        db = gutils.get_firestore_client_db()

        # Get the collection reference
        collection_ref = db.collection(collection_name)

        # Query the collection for documents with the specified player name and year
        query = collection_ref.where('player', '==', player_name).where('year', '==', year)
        docs = query.stream()

        results = []
        for doc in docs:
            if doc.exists:
                results.append(doc.to_dict())

        if not results:
            print(f"No documents found in {collection_name} for player {player_name} and year {year}")
            return None

        return results

    except Exception as e:
        print(f"Error querying {collection_name} for player {player_name} and year {year}: {e}")
        return None


In [25]:
query_firestore_player_data('Cade McNamara', 2022)[0].keys()

dict_keys(['sei', 'reer_competitive_rank', 'year', 'qb_total_score', 'qb_relative_score', 'ppi', 'ppi_competitive_rank', 'dmi', 'reer', 'pk', 'qb_total_rank', 'adpsr', 'defense_score_competitive_rank', 'ppi_relative_rank', 'team', 'dmi_competitive_rank', 'adpsr_relative_rank', 'player', 'crae', 'qpi', 'total_score_week_rank', 'crae_relative_rank', 'aqs_relative_rank', 'dmi_relative_rank', 'competitive_score_week_rank', 'defense_score', 'qpi_relative_rank', 'week', 'qpi_competitive_rank', 'crae_competitive_rank', 'sei_relative_rank', 'adpsr_competitive_rank', 'sei_competitive_rank', 'qb_competitive_score', 'defense_score_relative_rank', 'aqs', 'reer_relative_rank', 'aqs_competitive_rank'])

In [ ]:
week = 3
print('J.J. McCarthy', week, year)
player='J.J. McCarthy'
plays = dr.get_plays(year=year, team=team, week=week)
player_plays = plays[plays.play_text.str.contains(player, na=False)]
player_plays = player_plays[player_plays.offense==team]
opponent_team = player_plays.iloc[0]['defense']


In [ ]:
defense_df = pd.read_csv('/Users/djschor/Projects/ncaafb_power_rank/data/defense_2022/defense_2022.csv')

In [ ]:
# cqm.pressure_performance_index(player_plays)

In [ ]:

# PART 2 _______________________________
# gutils.batch_save_data_firestore(competitive_qb_metrics_df, "qb", 'pk')

# print("Saved QB performance metrics to GCP.")


In [ ]:
# sqm.calculate_qb_performance_metrics_save_gcp(2022)

In [ ]:
# get plays 
for week in range(1, 14):
    try:
        plays = dr.get_plays(year=year, team='Ohio State', week=week)
        player_plays = plays[plays.play_text.str.contains('C.J. Stroud', na=False)]
        player_plays = player_plays[player_plays.offense=='Ohio State']
        adpsr = cqm.pocket_presence_index(player_plays)
        print(adpsr)
    except: 
        print(" ")

In [ ]:
# function to get the week number given the team and opponent
def get_week_number(team, opponent_team, year):
    games = dr.get_games(year=year, team=team)
    week = games[(games.home_team == opponent_team) | (games.away_team == opponent_team)].week.max()
    return week

get_week_number('Tennessee', 'Alabama', year)

In [ ]:
player_plays['year'] = year

In [ ]:
team_roster = dr.get_team_rosters('Michigan', year)
qb_roster = team_roster[team_roster['position'] == 'QB']

if not qb_roster.empty:
    # Get player usage data for the QBs
    qb_usages = dr.get_player_usage(year, team='Michigan', position='QB')
    qb_usages = qb_usages[qb_usages['id'].isin(qb_roster['id'])].sort_values(by='usage_overall', ascending=False).head(2)


In [ ]:
games = dr.get_games(year=year, team='Arizona State')

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed

year = 2022
print(f"Calculating QB performance metrics for {year}...")

# Get all FBS teams
fbs_teams = dr.get_fbs_teams(season=year)
team_names = ['Michigan']


In [ ]:

def process_team_qb_data(team_name):
    print(f"Processing {team_name} QB data...")
    # Fetch team rosters and find the quarterbacks
    team_roster = dr.read_team_roster_csv(team_name, year)
    qb_roster = team_roster[team_roster['position'] == 'QB']
    if not qb_roster.empty:
        # Get player usage data for the QBs
        qb_usages = dr.read_qb_usage_csv(team_name)
        starting_qbs = qb_usages[qb_usages['id'].isin(qb_roster['id'])].sort_values(by='usage_overall', ascending=False).head(2)

        # Determine the starting quarterback(s)
        if starting_qbs.empty:
            # No dominant starter, get first and second string QBs by usage
            starting_qbs = qb_usages.nlargest(1, 'usage_overall')
        qb_data = []
        for _, starting_qb in starting_qbs.iterrows():
            if starting_qb['name'] is not None and starting_qb['team'] is not None:
                qb_name = starting_qb['name']
                qb_team = starting_qb['team']
                qb_df = sqm.get_qb_game_metrics_all_games(qb_name, qb_team, year).fillna(0)
                qb_df.to_csv('/Users/djschor/Projects/ncaafb_power_rank/data/qb_performance_2022/' + qb_name + '.csv')
                qb_data.append(qb_df)
        print(f"Completed {team_name} QB data")
        return pd.concat(qb_data, ignore_index=True) if qb_data else None
    else:
        return pd.DataFrame()

process_team_qb_data('Michigan')

In [28]:

def get_top_qbs_overall(top_x, field='avg_qb_total_score'):
    """
    Queries Firestore collection "qb_overall_rankings_2022" and returns the top X QBs based on the specified field.

    Args:
        top_x (int): The number of top QBs to return.
        field (str): The field name to sort the QBs by (e.g., 'avg_qb_competitive_score', 'avg_qb_total_score', 'competitive_score_rank', or 'total_score_rank').

    Returns:
        List[dict]: List of dictionaries containing top X QBs data.
    """
    if field not in {'avg_qb_competitive_score', 'avg_qb_total_score', 'competitive_score_rank', 'total_score_rank'}:
        raise ValueError("Invalid field. Valid fields are: 'avg_qb_competitive_score', 'avg_qb_total_score', 'competitive_score_rank', and 'total_score_rank'.")

    collection_name = "qb_overall_rankings_2022"

    # Initialize Firestore client
    db = gutils.get_firestore_client_db()

    # Query the Firestore collection, order by the specified field, and limit the results to top X
    query = db.collection(collection_name).order_by(field, direction=firestore.Query.DESCENDING if field.startswith('avg_') else firestore.Query.ASCENDING).limit(top_x).stream()

    # Retrieve all documents from the query
    top_qbs = [doc.to_dict() for doc in query]

    return top_qbs

def get_top_qbs_by_week(week, top_x):
    """
    Queries Firestore collection "qb_weekly_rankings_2022" and returns the top X QBs by week.
    
    Args:
        week (int): The week number to filter results.
        top_x (int): The number of top QBs to return.

    Returns:
        List[dict]: List of dictionaries containing top X QBs data.
    """
    collection_name = "qb_weekly_rankings_2022"

    # Initialize Firestore client
    db = gutils.get_firestore_client_db()

    # Query the Firestore collection for documents with the specified week and order by qb_total_score
    query = db.collection(collection_name).where("week", "==", week).order_by("qb_total_score", direction=firestore.Query.DESCENDING).limit(top_x).stream()

    # Retrieve all documents from the query
    top_qbs = [doc.to_dict() for doc in query]

    return top_qbs

get_top_qbs_by_week(5, 5)

[{'sei': 650.0,
  'reer_competitive_rank': 13.0,
  'year': 2022,
  'qb_total_score': 93.775774266017,
  'qb_relative_score': 93.80755114948556,
  'ppi': 50.24939211370077,
  'ppi_competitive_rank': 27.0,
  'dmi': 75.0,
  'reer': 100.0,
  'pk': 'Austin Aune_5_2022',
  'qb_total_rank': 25.0,
  'adpsr': 12.5,
  'defense_score_competitive_rank': 8.0,
  'ppi_relative_rank': 2.0,
  'team': 'North Texas',
  'dmi_competitive_rank': 96.0,
  'adpsr_relative_rank': 1.0,
  'player': 'Austin Aune',
  'crae': -25.36496673523676,
  'qpi': 259.2105263157894,
  'total_score_week_rank': 1.0,
  'crae_relative_rank': 8.0,
  'aqs_relative_rank': 10.0,
  'dmi_relative_rank': 11.0,
  'competitive_score_week_rank': 2.0,
  'defense_score': 61.30449486998842,
  'qpi_relative_rank': 4.0,
  'week': 5,
  'qpi_competitive_rank': 29.0,
  'crae_competitive_rank': 221.0,
  'sei_relative_rank': 2.0,
  'adpsr_competitive_rank': 15.0,
  'sei_competitive_rank': 45.0,
  'qb_competitive_score': 85.53640971541428,
  'defense

In [ ]:

def read_plays_csv(team, week):
    file_path = '/Users/djschor/Projects/ncaafb_power_rank/data/plays_2022/'
    file_name = f"{team}_{week}.csv"
    full_path = os.path.join(file_path, file_name)
    
    try:
        plays_df = pd.read_csv(full_path)
        return plays_df
    except FileNotFoundError as e:
        print(f"File not found: {e}")
        return None
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return None

def get_qb_game_metrics(player, team, week, year): 
    print(player, week, year)
    try:
        plays = read_plays_csv(team=team, week=week)#.drop(columns=['Unnamed: 0'])
        if 'play_text' in plays.columns:
            print('Getting play text')
            player_plays = plays[plays.play_text.str.contains(player, na=False)]
            player_plays = player_plays[player_plays.offense==team]

            if player_plays.empty:
                # Return a DataFrame with null values
                print('No plays for {} in week {} of {}'.format(player, week, year))
                return pd.DataFrame([[player, team, week, year, None, None, None, None, None, None, None, None, None]], columns=['player', 'team', 'week', 'year', 'aqs', 'qpi', 'sei', 'crae', 'dmi', 'ppi', 'adpsr', 'reer', 'defense_score'])

            # Get the opponent team
            opponent_team = player_plays.iloc[0]['defense']

            print('Getting defense')
            # Get the average defense_total_score of the opponent for weeks before the current week
            average_defense_score = sqm.get_opponent_defense_metrics(opponent_team, week, year)


            aqs = cqm.adaptive_quarterback_score(player_plays)
            qpi = cqm.quarterback_passing_index(player_plays)
            sei = cqm.scramble_efficiency_index(player_plays)
            crae = cqm.completion_rate_above_expected(player_plays)
            dmi = cqm.decision_making_index(player_plays)
            print('Getting PPI')
            ppi = cqm.pressure_performance_index(player_plays)
            adpsr = cqm.adjusted_deep_pass_success_rate(player_plays)
            reer = cqm.red_zone_efficiency_rating(player_plays)
            df = pd.DataFrame([[player, team, week, year, aqs, qpi, sei, crae, dmi, ppi, adpsr, reer, average_defense_score]], columns=['player', 'team', 'week', 'year', 'aqs', 'qpi', 'sei', 'crae', 'dmi', 'ppi', 'adpsr', 'reer', 'defense_score'])
        else:
            df = pd.DataFrame([[player, team, week, year, None, None, None, None, None, None, None, None, None]], columns=['player', 'team', 'week', 'year', 'aqs', 'qpi', 'sei', 'crae', 'dmi', 'ppi', 'adpsr', 'reer', 'defense_score'])
    except ValueError as e: 
        print('Error in QB Game Metrics: ', e)
        df = pd.DataFrame([[player, team, week, year, None, None, None, None, None, None, None, None, None]], columns=['player', 'team', 'week', 'year', 'aqs', 'qpi', 'sei', 'crae', 'dmi', 'ppi', 'adpsr', 'reer', 'defense_score'])
    return df 

In [ ]:
def read_plays_csv(team, week):
    file_path = '/Users/djschor/Projects/ncaafb_power_rank/data/plays_2022/'
    file_name = f"{team}_{week}.csv"
    full_path = os.path.join(file_path, file_name)
    
    try:
        plays_df = pd.read_csv(full_path)
        return plays_df
    except FileNotFoundError as e:
        print(f"File not found: {e}")
        return None
    except Exception as e:
        print(f"Error reading CSV: {e}")
        return None
read_plays_csv(team, 1).columns

In [ ]:
get_qb_game_metrics('C.J. Stroud', 'Ohio State', 2, 2022)

In [ ]:
year=2022
print(f"Calculating QB performance metrics for {year}...")

# Get all FBS teams
fbs_teams = dr.get_fbs_teams(season=year)
team_names = fbs_teams['school'].tolist()
# team_names = ['Michigan']

def process_team_qb_data(team_name):
    print(f"Processing {team_name} QB data...")
    # Fetch team rosters and find the quarterbacks
    team_roster = dr.read_team_roster_csv(team_name, year)
    qb_roster = team_roster[team_roster['position'] == 'QB']
    if not qb_roster.empty:
        # Get player usage data for the QBs
        qb_usages = dr.read_qb_usage_csv(team_name)
        starting_qbs = qb_usages[qb_usages['id'].isin(qb_roster['id'])].sort_values(by='usage_overall', ascending=False).head(2)

        # Determine the starting quarterback(s)
        if starting_qbs.empty:
            # No dominant starter, get first and second string QBs by usage
            starting_qbs = qb_usages.nlargest(1, 'usage_overall')
        qb_data = []
        for _, starting_qb in starting_qbs.iterrows():
            if starting_qb['name'] is not None and starting_qb['team'] is not None:
                qb_name = starting_qb['name']
                qb_team = starting_qb['team']
                qb_df = sqm.get_qb_game_metrics_all_games(qb_name, qb_team, year).fillna(0)
                qb_df.to_csv('/Users/djschor/Projects/ncaafb_power_rank/data/qb_performance_2022/' + qb_name + '.csv')
                qb_data.append(qb_df)
        print(f"Completed {team_name} QB data")
        return pd.concat(qb_data, ignore_index=True) if qb_data else None
    else:
        return pd.DataFrame()


with ThreadPoolExecutor(max_workers=5) as executor:
    futures = [executor.submit(process_team_qb_data, team_name) for team_name in team_names]
    results = [future.result() for future in as_completed(futures)]
all_qb_data_df = pd.concat(results)

# save player names to meta collection gcp 
qb_names = all_qb_data_df['player'].unique().tolist()
gutils.save_data_to_firestore_id({'qbs': qb_names}, gutils.get_firestore_client_db(), 'meta', 'qbs')

# Calculate relative metrics for each QB
relative_qb_metrics = sqm.calculate_qb_relative_metrics(all_qb_data_df)

# Get the list of unique weeks in the DataFrame
unique_weeks = relative_qb_metrics['week'].unique()

# Execute calculate_qb_week_competitive_metrics concurrently using a ThreadPoolExecutor
with ThreadPoolExecutor(max_workers=5) as executor:
    week_competitive_metrics_futures = [executor.submit(functools.partial(sqm.calculate_qb_week_competitive_metrics, relative_qb_metrics), week) for week in unique_weeks]

    week_competitive_metrics = []
    for future in as_completed(week_competitive_metrics_futures):
        week_competitive_metrics.append(future.result())

competitive_qb_metrics_df = pd.concat(week_competitive_metrics)

# Calculate the total defense score
qb_df = sqm.score_qb_total(competitive_qb_metrics_df)

# Save the data to a GCP collection called "qb_performance"
print("Saving QB performance metrics to GCP...")
qb_df['pk'] = qb_df['player'] + '_' + qb_df['week'].astype(str) + '_' + qb_df['year'].astype(str)

In [ ]:
qb_df['pk']

In [ ]:
gutils.batch_save_data_firestore(qb_df.reset_index(drop=True), "qb", 'pk')


In [ ]:
gutils.query_firestore_general('qb_weekly_rankings_2022', 'C.J. Stroud_1_2022')

In [ ]:
get_qb_by_name('C.J. Stroud')

In [ ]:
import pandas as pd
from typing import List

def calculate_save_overall_qb_rankings(qb_df):

    # Get list of QBs from the meta collection
    # qbs = gutils.query_firestore_general('meta', f"qbs")['qbs']
    qbs = qb_df.player.unique().tolist()
    overall_qb_rankings = []

    for qb in qbs:
        print(f"Calculating avg QB rankings for {qb}...")

        # Get weekly QB performance data for the specified year
        # qb_performance_data = gutils.query_firestore_player_data('qb', qb, year)
        qb_performance_data = qb_df[qb_df.player==qb]
        if not qb_performance_data.empty:
            # Calculate the average QB total score and competitive score for the season
            qb_total_scores = [row['qb_total_score'] for _, row in qb_performance_data.iterrows()]
            qb_competitive_scores = [row['qb_competitive_score'] for _, row in qb_performance_data.iterrows()]
            avg_qb_total_score = sum(qb_total_scores) / len(qb_total_scores)
            avg_qb_competitive_score = sum(qb_competitive_scores) / len(qb_competitive_scores)

            overall_qb_rankings.append({
                'player': qb,
                'year': year,
                'avg_qb_total_score': avg_qb_total_score,
                'avg_qb_competitive_score': avg_qb_competitive_score
            })

    # Convert the list of dictionaries to a DataFrame
    overall_qb_rankings_df = pd.DataFrame(overall_qb_rankings)

    # Calculate the rankings based on the average QB total score and competitive score
    overall_qb_rankings_df['total_score_rank'] = overall_qb_rankings_df['avg_qb_total_score'].rank(ascending=False)
    overall_qb_rankings_df['competitive_score_rank'] = overall_qb_rankings_df['avg_qb_competitive_score'].rank(ascending=False)

    # Save the overall QB rankings to Firestore
    gutils.batch_save_data_firestore(overall_qb_rankings_df, f"qb_overall_rankings_{year}", id='player')
    print(f"Saved overall QB rankings for {year} to Firestore.")
    return


def calculate_save_weekly_qb_rankings(qb_df):
    weeks = qb_df['week'].unique().tolist()
    # Iterate through weeks
    for week in weeks:  # Assuming a 15-week season
        # Filter the all_qb_data DataFrame for the specified week
        weekly_qb_data = qb_df[qb_df['week'] == week]

        # Check if weekly_qb_data is not empty before calculating rankings
        if not weekly_qb_data.empty:
            # Calculate the rankings based on the QB total score and competitive score for the week
            weekly_qb_data.loc[:, 'total_score_week_rank'] = weekly_qb_data['qb_total_score'].rank(ascending=False)
            weekly_qb_data.loc[:, 'competitive_score_week_rank'] = weekly_qb_data['qb_competitive_score'].rank(ascending=False)

            # Save the weekly QB rankings to Firestore
            gutils.batch_save_data_firestore(weekly_qb_data, f"qb_weekly_rankings_{year}", id='pk')
        else:
            print(f"No QB data available for week {week}.")

def drive_weekly_and_overall_qb_rankings(qb_df):
    calculate_save_overall_qb_rankings(qb_df)
    calculate_save_weekly_qb_rankings(qb_df)
    print(f"Saved weekly and overall QB rankings for {year} to Firestore.")
    return


def get_top_qbs_overall(top_x, field='avg_qb_total_score'):
    """
    Queries Firestore collection "qb_overall_rankings_2022" and returns the top X QBs based on the specified field.

    Args:
        top_x (int): The number of top QBs to return.
        field (str): The field name to sort the QBs by (e.g., 'avg_qb_competitive_score', 'avg_qb_total_score', 'competitive_score_rank', or 'total_score_rank').

    Returns:
        List[dict]: List of dictionaries containing top X QBs data.
    """
    if field not in {'avg_qb_competitive_score', 'avg_qb_total_score', 'competitive_score_rank', 'total_score_rank'}:
        raise ValueError("Invalid field. Valid fields are: 'avg_qb_competitive_score', 'avg_qb_total_score', 'competitive_score_rank', and 'total_score_rank'.")

    collection_name = "qb_overall_rankings_2022"

    # Initialize Firestore client
    db = gutils.get_firestore_client_db()

    # Query the Firestore collection, order by the specified field, and limit the results to top X
    query = db.collection(collection_name).order_by(field, direction=firestore.Query.DESCENDING if field.startswith('avg_') else firestore.Query.ASCENDING).limit(top_x).stream()

    # Retrieve all documents from the query
    top_qbs = [doc.to_dict() for doc in query]

    return top_qbs

def get_top_qbs_by_week(week, top_x):
    """
    Queries Firestore collection "qb_weekly_rankings_2022" and returns the top X QBs by week.
    
    Args:
        week (int): The week number to filter results.
        top_x (int): The number of top QBs to return.

    Returns:
        List[dict]: List of dictionaries containing top X QBs data.
    """
    collection_name = "qb_weekly_rankings_2022"

    # Initialize Firestore client
    db = gutils.get_firestore_client_db()

    # Query the Firestore collection for documents with the specified week and order by qb_total_score
    query = db.collection(collection_name).where("week", "==", week).order_by("qb_total_score", direction=firestore.Query.DESCENDING).limit(top_x).stream()

    # Retrieve all documents from the query
    top_qbs = [doc.to_dict() for doc in query]

    return pd.DataFrame(top_qbs)


# Example usage:
# top_5_qbs = get_top_qbs_by_week(12, 5)
# top_5_qbs

get_top_qbs_overall(5)

In [ ]:
i have a firestore collection called qb_overall_rankings_2022 where documents are formatted with the identifier <qb name>. The items in the document are formatted like: avg_qb_competitive_score:73, avg_qb_total_score: 66, competitive_score_rank:140, player:"AJ Mayer",total_score_rank:142,year:2022. please create a function to query the collection and return the collection to get the top x qbs by avg_qb_competitive_score
,avg_qb_total_score, competitive_score_rank, or total_score_rank


In [ ]:
{avg_qb_competitive_score:73, avg_qb_total_score: 66, competitive_score_rank:140, player:"AJ Mayer",total_score_rank:142,year:2022}

In [ ]:
query_overall_qb_rankings(2022)

In [ ]:
all_qb_data[all_qb_data['week'] == 1]

In [ ]:
qb_performance_data = pd.DataFrame(gutils.query_firestore_player_data('qb', 'Hendon Hooker', year))

In [ ]:
from concurrent.futures import ThreadPoolExecutor
import functools
import pandas as pd

year = 2022
qbs = gutils.query_firestore_general('meta', f"qbs")['qbs']

def fetch_qb_data(year, qb):
    qb_performance_data = pd.DataFrame(gutils.query_firestore_player_data('qb', qb, year))  # Replaced with the correct `qb`
    
    if qb_performance_data is not None and not qb_performance_data.empty:
        print(f"QB: {qb}, Good")
        return qb_performance_data
    return None

# Get all QB performance data for the specified year and concatenate into a single DataFrame
with ThreadPoolExecutor() as executor:
    qb_data_frames = [df for df in executor.map(functools.partial(fetch_qb_data, year), qbs) if df is not None]

# Ensure there is at least one DataFrame in qb_data_frames before concatenating
if qb_data_frames:
    all_qb_data = pd.concat(qb_data_frames)

   
else:
    print("No QB data available for concatenation.")



In [ ]:
calculate_save_weekly_qb_rankings(2022)

In [ ]:
Based on the qb scoring functions and data saved in the qb database for 2022 qb performance, create functions for the following: 
1. calculate_save_overall_qb_rankings, averages the weekly qb performance scores to get the scores and rankings for each quarterback over the whole season. once all qbs are calculated, save the results a firestore collection called qb_overall_rankings_2022
2. calculate_weekly_qb_rankings, for each week in the season, get all qbs who performed and get scores and rankings for the qbs that week . once all qbs are calculated, save the results a firestore collection called qb_weekly_rankings_2022
3. query functions corresponding to the qb overall and weekly collections saved in the two previous functions 
4. query functions to get rankings by conference for the year

Here are the rules to remember: 
1. Make sure to use football gutils functions as applicable
2. the qb performance data is located in the collection 'qb' and can be accessed with the function query_firestore_player_data
Here is that function again:
def query_firestore_player_data(collection_name, player_name, year):
    try:
        # Initialize Firestore client
        db = gutils.get_firestore_client_db()

        # Get the collection reference
        collection_ref = db.collection(collection_name)

        # Query the collection for documents with the specified player name and year
        query = collection_ref.where('player', '==', player_name).where('year', '==', year)
        docs = query.stream()

        results = []
        for doc in docs:
            if doc.exists:
                results.append(doc.to_dict())

        if not results:
            print(f"No documents found in {collection_name} for player {player_name} and year {year}")
            return None

        return results

    except Exception as e:
        print(f"Error querying {collection_name} for player {player_name} and year {year}: {e}")
       

3. the new functions should save to qb_weekly_rankings_2022 and qb_overall_rankings_2022 collections 
4. there is no collection named f"qb_performance_{year}". To get a list of all qbs query the meta collection for the qbs id, or to get a specific team's qbs during a year then query the meta table for the id <team>_<year>
5. Make sure to follow the gutils functions accurately. Here they are again. It is of the UTMOST IMPORTANCE you do not get the function names wrong. They are all to be imported with gutils.<function>. Here are the functions again: 
6. Make sure to adhere to the data in the gcp collections accurately. Do not assume. If you do not know, ask me. Here is what the query_firestore_player_data function returns for JJ Mccarthy in 2022. I am sending it to you in to_dict('records') format so you can read it but assume that the function returns it in DATAFRAME FORMAT! Here is the data: [{'sei': 8.0, 'reer_competitive_rank': 12.0, 'year': 2022, 'qb_total_score': 73.49521783101324, 'qb_relative_score': 71.69927976481077, 'reer': 60.0, 'tid': 0.0, 'dmi': 0.8571428571428571, 'bpr': 6.451612903225806, 'defense_score_relative_rank': 8.0, 'qb_total_rank': 648.0, 'pk': 'J.J. McCarthy_10_2022', 'defense_score_competitive_rank': 109.0, 'tid_competitive_rank': 123.0, 'team': 'Michigan', 'dmi_competitive_rank': 51.0, 'player': 'J.J. McCarthy', 'crae': -0.24112305949580154, 'qpi': 1.5370370370370372, 'crae_relative_rank': 7.0, 'aqs_relative_rank': 7.0, 'dmi_relative_rank': 8.0, 'bpr_relative_rank': 8.0, 'defense_score': 51.9787115298463, 'qpi_relative_rank': 9.0, 'week': 10, 'qpi_competitive_rank': 73.0, 'crae_competitive_rank': 108.0, 'sei_relative_rank': 2.0, 'bpr_competitive_rank': 62.0, 'sei_competitive_rank': 25.0, 'qb_competitive_score': 75.21591937270145, 'tid_relative_rank': 9.0, 'aqs': 0.5138888888888888, 'reer_relative_rank': 2.0, 'aqs_competitive_rank': 51.0}, {'sei': 0.0, 'reer_competitive_rank': 7.0, 'year': 2022, 'qb_total_score': 80.58947187293148, 'qb_relative_score': 72.24474687653978, 'reer': 100.0, 'tid': 0.0, 'dmi': 0.6, 'bpr': 10.0, 'defense_score_relative_rank': 4.0, 'qb_total_rank': 257.0, 'pk': 'J.J. McCarthy_11_2022', 'defense_score_competitive_rank': 58.0, 'tid_competitive_rank': 123.0, 'team': 'Michigan', 'dmi_competitive_rank': 86.0, 'player': 'J.J. McCarthy', 'crae': -0.32279666500174387, 'qpi': 1.6176470588235294, 'crae_relative_rank': 9.0, 'aqs_relative_rank': 4.0, 'dmi_relative_rank': 9.0, 'bpr_relative_rank': 6.0, 'defense_score': 54.48354547212146, 'qpi_relative_rank': 8.0, 'week': 11, 'qpi_competitive_rank': 61.0, 'crae_competitive_rank': 116.0, 'sei_relative_rank': 8.0, 'bpr_competitive_rank': 30.0, 'sei_competitive_rank': 120.0, 'qb_competitive_score': 78.48699525176522, 'tid_relative_rank': 9.0, 'aqs': 0.6470588235294118, 'reer_relative_rank': 1.0, 'aqs_competitive_rank': 14.0}]


In [ ]:
jj = pd.DataFrame(gutils.query_firestore_player_data('qb', 'J.J. McCarthy', 2022))

In [ ]:
jj

In [ ]:
print(jj.head(2).to_dict('records'))

In [ ]:
jj.columns

In [ ]:
jj['week']

In [ ]:
collection_name, name, week, year = 'qb', 'J.J. McCarthy', 1, 2022
pk = '{}_{}_{}'.format(name, week, year)
# pk = 'AJ Mayer_11_2022'
df = query_firestore_general('qb', pk)

In [ ]:
df

In [ ]:
collection_name, name, week, year = 'qb', 'J.J. McCarthy', 1, 2022
name = query_firestore_general('meta', 'Michigan_2022')['player'][0]
db = gutils.get_firestore_client_db()
pk = f"{name}_{week}_{year}"
# Define a function to query data for collections with the new document ID format
def query_new_id_format(collection, start_date=None, end_date=None):
    if start_date and end_date:
        query = collection.where('pk', '==', pk).where('date', '>=', start_date).where('date', '<=', end_date)
    else:
        query = collection.where('pk', '==', pk)

    docs = query.stream()
    return {doc.id: doc.to_dict() for doc in docs}

# Get the collection reference
collection_ref = db.collection(collection_name)

doc_ref = collection_ref.document(pk)
df =  doc_ref.get().to_dict() 

In [ ]:
df

In [ ]:
gutils.query_firestore_player('qb', 'J.J. McCarthy', 1, 2022)

### Preprocessing:
 - Clean and preprocess the collected data.
 - Handle missing values, outliers, and convert categorical data into numerical form.


In [ ]:

# Convert the data into a DataFrame
team_df = pd.DataFrame(teams)

# Drop unnecessary columns
team_df = team_df.drop(columns=['id', 'abbreviation'])

# Encode categorical columns
team_df['conference'] = team_df['conference'].astype('category')
team_df['conference_code'] = team_df['conference'].cat.codes

### Feature Engineering:
 - Create new features from existing data.
 - Normalize or standardize the data if required.


In [ ]:
# Example: Add winning percentage
team_df['winning_percentage'] = team_df['wins'] / (team_df['wins'] + team_df['losses'])

# Standardize numerical features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
team_df[['wins', 'losses', 'winning_percentage']] = scaler.fit_transform(team_df[['wins', 'losses', 'winning_percentage']])


### Model Training:
 - Train a machine learning model to rank teams based on their features.
 - Example: Use a clustering algorithm like KMeans to group similar teams together.


In [ ]:
from sklearn.cluster import KMeans

# Select features for clustering
features = ['conference_code', 'wins', 'losses', 'winning_percentage']
X = team_df[features]

# Train KMeans model
kmeans = KMeans(n_clusters=5, random_state=42)
kmeans.fit(X)

# Add cluster labels to DataFrame
team_df['cluster'] = kmeans.labels_
